In [1]:
import requests
import time

restaurants = []  # 배열 선언

# 전체 페이지 수를 설정
total_pages = 576
url_template = "https://www.bluer.co.kr/api/v1/restaurants?page={page}&size=30&query=&foodType=&foodTypeDetail=&feature=&location=&locationDetail=&area=&areaDetail=&priceRange=&ribbonType=&recommended=false&isSearchName=false&tabMode=single&searchMode=ribbonType&zone1=&zone2=&zone2Lat=&zone2Lng="

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept": "application/hal+json",
    "x-requested-with": "XMLHttpRequest"
}

# 크롤링 진행
for page in range(total_pages):
    url = url_template.format(page=page)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        restaurants.extend(data["_embedded"]["restaurants"])
        print(f"Page {page + 1}/{total_pages} collected successfully.")
    else:
        print(f"Failed to fetch page {page + 1}. Status code: {response.status_code}")
    
    # 각 요청 사이에 시간 간격 추가
    time.sleep(2)  # 2초 간격

# 전체 데이터를 저장
import json
with open("restaurants.json", "w", encoding="utf-8") as f:
    json.dump(restaurants, f, ensure_ascii=False, indent=4)

print("전체 페이지 크롤링 완료. JSON 파일로 저장됨.")


Page 1/576 collected successfully.
Page 2/576 collected successfully.
Page 3/576 collected successfully.
Page 4/576 collected successfully.
Page 5/576 collected successfully.
Page 6/576 collected successfully.
Page 7/576 collected successfully.
Page 8/576 collected successfully.
Page 9/576 collected successfully.
Page 10/576 collected successfully.
Page 11/576 collected successfully.
Page 12/576 collected successfully.
Page 13/576 collected successfully.
Page 14/576 collected successfully.
Page 15/576 collected successfully.
Page 16/576 collected successfully.
Page 17/576 collected successfully.
Page 18/576 collected successfully.
Page 19/576 collected successfully.
Page 20/576 collected successfully.
Page 21/576 collected successfully.
Page 22/576 collected successfully.
Page 23/576 collected successfully.
Page 24/576 collected successfully.
Page 25/576 collected successfully.
Page 26/576 collected successfully.
Page 27/576 collected successfully.
Page 28/576 collected successfully.
P

In [27]:
#### 1. 전처리

import pandas as pd
import json

# JSON 파일 읽기
with open("restaurants.json", "r", encoding="utf-8") as f:
    restaurants = json.load(f)

# Pandas DataFrame으로 변환
df = pd.DataFrame(restaurants)

# 기존 전처리 코드 적용
# 필요 없는 컬럼 제거
columns_to_drop = ["createdDate", "timeInfo", "gps", "tags", "status",
                   "buzUsername", "business", "pageView", "brandMatchStatus", "brandRejectReason",
                   "orderDescending", "foodTypeDetails","foodDetailTypes", "countEvaluate", "bookmark", "features",
                   "feature107", "brandBranches", "brandHead", "firstImage", "firstLogoImage",
                   "_links"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Nested JSON 컬럼 정규화
nested_columns = ["headerInfo", "defaultInfo", "statusInfo", "juso", "review", "etcInfo"]
for column in nested_columns:
    if column in df.columns:
        n = pd.json_normalize(df[column])  # 정규화
        n.columns = [f"{column}_{subcol}" for subcol in n.columns]  # 열 이름 접두사 추가
        df = pd.concat([df.drop(columns=[column]), n], axis=1)  # 기존 열 삭제 후 결합
if "foodTypes" in df.columns:
    df["foodTypes"] = df["foodTypes"].apply(lambda x: ", ".join(x) if isinstance(x, list) else None)
else:
    print("foodTypes 컬럼이 누락되었습니다. 데이터 확인 필요.")
# 필요 없는 sub_columns 제거
sub_columns_to_drop = ["headerInfo_nickname", "headerInfo_year", "headerInfo_ribbonTypeByOrdinal", "headerInfo_nameEN", "headerInfo_nameCN", "headerInfo_bookYear",
                       "defaultInfo_websiteFacebook", "defaultInfo_chefName", "statusInfo_storeType", "statusInfo_openEra", "statusInfo_openDate",

                       "juso_roadAddrPart2", "juso_jibunAddr", "juso_zipNo",
                       "juso_admCd", "juso_bdNm", "juso_buldMnnm", "juso_buldSlno", "juso_detBdNmList", "juso_zone2_1", "juso_zone2_2", "juso_map_1",
                       "juso_map_2", "juso_detailAddress" , "juso_emdNm", "juso_engAddr", "juso_liNm",  
                       "review_readerReview", "review_businessReview", "review_editorReview",
                       "etcInfo_toilet", "etcInfo_toiletEtc", "etcInfo_close", "etcInfo_interior",
                       "etcInfo_appYn", "etcInfo_projectNo", "etcInfo_reviewerRecommend", "etcInfo_onlySiteView",
                       "etcInfo_history", "etcInfo_mainMemo"]
df = df.drop(columns=[col for col in sub_columns_to_drop if col in df.columns])

# 데이터 저장
df.to_csv("blueRibbon.csv", index=False)
print("전처리 테스트 완료. CSV 파일 저장됨.")


전처리 테스트 완료. CSV 파일 저장됨.


In [29]:
data = pd.read_csv("blueRibbon.csv")
print(data.columns)
data.iloc[20:40, 0:5]

Index(['id', 'bookStatus', 'foodTypes', 'headerInfo_nameKR',
       'headerInfo_ribbonType', 'defaultInfo_website',
       'defaultInfo_websiteInstagram', 'defaultInfo_phone',
       'defaultInfo_openHours', 'defaultInfo_closeHours',
       'defaultInfo_openHoursWeekend', 'defaultInfo_closeHoursWeekend',
       'defaultInfo_dayOff', 'defaultInfo_app2Yn', 'statusInfo_parking',
       'statusInfo_creditCard', 'statusInfo_visit', 'statusInfo_menu',
       'statusInfo_priceRange', 'statusInfo_businessHours',
       'statusInfo_newOpenDate', 'juso_roadAddrPart1', 'juso_siNm',
       'juso_sggNm', 'juso_rn', 'review_review', 'review_reviewSimple',
       'review_readerAppraisal', 'etcInfo_seat', 'etcInfo_room',
       'etcInfo_chain', 'etcInfo_renewal', 'etcInfo_head', 'etcInfo_branch'],
      dtype='object')


C:\Users\Playdata\AppData\Local\Temp\ipykernel_16812\2552257997.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("blueRibbon.csv")


,id,bookStatus,foodTypes,headerInfo_nameKR,headerInfo_ribbonType
20,29206,APPROVAL,"일식, 모던재패니즈, 컨템포러리",고료리켄,RIBBON_THREE
21,28236,APPROVAL,"한식(육류), 한우오마카세, 소고기구이",본앤브레드,RIBBON_THREE
22,28229,APPROVAL,컨템포러리,알라프리마,RIBBON_THREE
23,28167,APPROVAL,"한식(일반한식), 모던한식",권숙수,RIBBON_THREE
24,26017,APPROVAL,"컨템포러리, 프랑스식",제로컴플렉스,RIBBON_THREE
25,20038,APPROVAL,이탈리아식,테이블포포,RIBBON_THREE
26,15931,APPROVAL,프랑스식,파씨오네,RIBBON_THREE
27,4953,APPROVAL,프랑스식,비스트로드욘트빌,RIBBON_THREE
28,4945,APPROVAL,"한식(일반한식), 모던한식",무궁화,RIBBON_THREE
29,4667,APPROVAL,"한식(일반한식), 모던한식, 뉴코리안",정식당,RIBBON_THREE


In [30]:
import numpy as np
import re

# 빈칸이나 "없음"으로 표기된 칸을 None으로 변경
df.replace(["", "없음"], None, inplace=True)

# foodDetailTypes의 리스트 해제
# 각 행의 값이 리스트라면 이를 문자열로 변환 (쉼표로 구분된 문자열로 병합)
if "foodTypes" in df.columns:
    df["foodTypes"] = df["foodTypes"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


# 웹사이트 열 통합
if "defaultInfo_website" in df.columns and "defaultInfo_websiteInstagram" in df.columns:
    def merge_websites(row):
        websites = [site for site in [row.get("defaultInfo_website"), row.get("defaultInfo_websiteInstagram")] if site]
        return ", ".join(websites) if websites else None

    df["defaultInfo_website_combined"] = df.apply(merge_websites, axis=1)
    df.drop(columns=["defaultInfo_website", "defaultInfo_websiteInstagram"], inplace=True)
    df.rename(columns={"defaultInfo_website_combined": "defaultInfo_website"}, inplace=True)

# statusInfo_openDate 형식 변환 (년도 4자리로 추출 후 "2024년" 형식으로 표기)
def extract_year_with_suffix(date):
    if isinstance(date, str):
        # 패턴 매칭으로 숫자 4자리 추출
        match = re.search(r'\d{4}', date)
        if match:
            return f"{int(match.group(0))}년"  # "2024년" 형식으로 반환
    return None

if "statusInfo_openDate" in df.columns:
    df["statusInfo_openDate"] = df["statusInfo_openDate"].apply(extract_year_with_suffix)

# 최종 결과를 CSV 파일로 저장
df.to_csv("cleaned_all_restaurants.csv", index=False)
print("수정된 최종 CSV 파일 저장 완료!")


수정된 최종 CSV 파일 저장 완료!


In [31]:
temp = pd.read_csv("cleaned_all_restaurants.csv")
temp

C:\Users\Playdata\AppData\Local\Temp\ipykernel_16812\1306026117.py:1: DtypeWarning: Columns (8,9,27) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv("cleaned_all_restaurants.csv")


,id,bookStatus,foodTypes,headerInfo_nameKR,headerInfo_ribbonType,defaultInfo_phone,defaultInfo_openHours,defaultInfo_closeHours,defaultInfo_openHoursWeekend,defaultInfo_closeHoursWeekend,...,review_review,review_reviewSimple,review_readerAppraisal,etcInfo_seat,etcInfo_room,etcInfo_chain,etcInfo_renewal,etcInfo_head,etcInfo_branch,defaultInfo_website
0,4627,APPROVAL,"중식, 일반중식",홍보각,RIBBON_THREE,02-531-6479,NaN,NaN,NaN,NaN,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,NaN,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,NaN,NaN,https://www.instagram.com/hongbogak_official/
1,29083,APPROVAL,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,RIBBON_THREE,02-518-0318,12:00,22:30,NaN,NaN,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,NaN,NaN,False,False,NaN,NaN,https://www.instagram.com/choidot_official/
2,28946,APPROVAL,프랑스식,더그린테이블,RIBBON_THREE,02-591-2672,NaN,NaN,NaN,NaN,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,NaN,NaN,False,False,NaN,NaN,"http://thegreentable.co.kr/, https://www.insta..."
3,4963,APPROVAL,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,RIBBON_THREE,02-6330-9050,NaN,NaN,NaN,NaN,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",NaN,300,4개,False,False,NaN,NaN,https://www.walkerhill.com/grandwalkerhillseou...
4,859,APPROVAL,프랑스식,콘티넨탈,RIBBON_THREE,02-2230-3369,NaN,NaN,NaN,NaN,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,NaN,NaN,https://www.shilla.net/seoul/dining/viewDining...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,25891,INACTIVE,"한식(어패류), 생선회",8호단골횟집,NOT_RECORD,032-885-9955,10:00,22:00,NaN,NaN,...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,NaN,NaN,NaN,False,False,NaN,NaN,NaN
17276,26401,INACTIVE,"일식, 스시",키무스시,NOT_RECORD,031-411-0603,12:00,22:00,NaN,NaN,...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,NaN,NaN,NaN,False,False,NaN,NaN,NaN
17277,26400,INACTIVE,"카페/커피/티, 카페",카페바다,NOT_RECORD,010-3713-3256,10:00,19:00,NaN,NaN,...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,NaN,NaN,NaN,False,False,NaN,NaN,"https://cafebada.modoo.at/, https://www.instag..."
17278,26203,INACTIVE,"디저트/베이커리, 떡",미시루,NOT_RECORD,042-825-9902,06:00,19:00,NaN,NaN,...,"현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...","현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...",NaN,NaN,NaN,False,False,NaN,진관점(02-388-7710),http://www.미시루.com/


In [5]:
import os
import json
import time
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("BASE_URL")
headers = json.loads(os.getenv("HEADERS"))

# 크롤링 데이터 저장할 리스트
restaurants = []


cnt = 0
# 데이터 크롤링
for page in range(0, 2):
    url = f"{base_url}?page={page}&size=30&query=&foodType=&foodTypeDetail=&feature=&location=&locationDetail=&area=&areaDetail=&priceRange=&ribbonType=&recommended=false&isSearchName=false&tabMode=single&searchMode=ribbonType&zone1=&zone2=&zone2Lat=&zone2Lng="
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            cnt += 1
            data = response.json()
            if "_embedded" in data and "restaurants" in data["_embedded"]:
                restaurants.extend(data["_embedded"]["restaurants"])
            print(f"Page {page} 크롤링 성공공")
        else:
            print(f"크롤링 실패패 {page}: {response.status_code}")
        
        if cnt % 100 == 0:
            time.sleep(5)
    except Exception as e:
        print(f"페이지 오류 {page}: {e}")
        break

# DataFrame으로 변환 후 CSV 저장
if restaurants:
    df = pd.DataFrame(restaurants)
    df.to_csv("restaurants_data.csv", index=False, encoding="utf-8-sig")
    print("Data saved to restaurants_data.csv")
else:
    print("No data collected.")


Page 0 크롤링 성공공
Page 1 크롤링 성공공
Data saved to restaurants_data.csv
